In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import Dataset
from utils import DropDatasetDuplicate

# 1. 处理预训练阶段数据
## 1.1 处理Wiki数据

In [2]:
origin_wiki_file = './data/wiki.simple.txt'
tokenizer_dir = './model_save/tokenizer/'
liness = []
with open(origin_wiki_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
# 如果词表大小小于 65535 用uint16存储，节省磁盘空间，否则用uint32存储
ids_dtype = np.uint16 if (len(tokenizer) // 64 + 1) * 64 < 65535 else np.uint32

In [4]:
lines[0:5]

['数学：\n',
 '数学，是研究数量、结构以及空间等概念及其变化的一门学科，属于形式科学的一种。数学利用抽象化和逻辑推理，从计数、计算、量度、对物体形状及运动的观察发展而成。数学家们拓展这些概念，以公式化新的猜想，以及从选定的公理及定义出发，严谨地推导出一些定理。\n',
 '基础数学的知识与运用是生活中不可或缺的一环。对数学基本概念的完善，早在古埃及、美索不达米亚及古印度历史上的古代数学文本便可观见，而在古希腊那里有更为严谨的处理。从那时开始，数学的发展便持续不断地小幅进展，至16世纪的文艺复兴时期，因为新的科学发现和数学革新两者的交互，致使数学的加速发展，直至今日。数学并成为许多国家及地区的教育中的一部分。\n',
 '数学在许多领域都有应用，包括科学、工程、医学、经济学和金融学等。数学对这些领域的应用通常被称为应用数学，有时亦会激起新的数学发现，并导致全新学科的发展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。数学家也研究纯粹数学，就是数学本身的实质性内容，而不以任何实际应用为目标。许多研究虽然以纯粹数学开始，但其过程中也发现许多可用之处。\n',
 '词源。\n']

In [ ]:
#去重
# dataset_duplicate = DropDatasetDuplicate(threshold=0.90, num_perm=128)
# for i, line in tqdm(enumerate(lines), total=len(lines)):
#     doc = line.strip()

#     if len(doc) != 0:
#         dataset_duplicate.add_doc(index=i, doc=doc)

# duplicate_set = dataset_duplicate.get_duplicate_indexs()

# no_duplicate_lines = [lines[i] for i in range(len(lines)) if i not in duplicate_set]

In [5]:
len(lines)

10103707

合并词条和内容

In [6]:
items, content = [], []
key_word, kw_line_idx = '', 0
content_start = False  # 词条内容开始标记

for i, line in enumerate(lines):
    
    line_strip = line.strip()

    # 词条以冒号`：`结尾
    if len(line_strip) > 0 and line_strip[-1] in (':', '：'):
        key_word = ''.join(line_strip[: -1])
        kw_line_idx = i 
        continue
    
    # 词条key_word在下一行，则合并上个词条并保存
    if i == kw_line_idx + 1 and key_word in line_strip or i == len(lines) - 1:
        txt = ''.join(content)

        if len(txt) > 0:
            items.append(txt)
            
        content = []
        content.append(f"{key_word}：")
    
    content.append(line)


In [7]:
len(items)

1008067

In [8]:
items[20]

'地质学：地质学（法语、德语：Geologie；拉丁语、西班牙语：Geologia；源于希腊语，γῆ，和，λoγία）是对地球的起源、历史与结构进行研究的学科。主要研究地球的物质组成、内部构造、外部特征、各圈层间的相互作用和演变历史。在现阶段，由于观察、研究条件的限制，主要以岩石圈为研究对象，并涉及水圈、大气圈、生物圈和岩石圈下更深的部分，以及涉及其他行星和卫星的太空地质学。\n地球历史学。\n-zh-hant：zh-hans\n地质时间尺度涵盖了整个地球历史。其起点开始于最早的太阳系材料生成时，约在4。567Ga，地球的出现约在4。54，Ga，一开始称为冥古宙，最年轻的地质年代则是全新世。\n地质年代比例。\n如果把地球诞生到现在的大约45亿年缩小到1年，则人类（Homo，apiens存在的时间只有极短的两分钟。\n显生宙。\n地质学学史。\n对地球的物质成分的研究最早至少可以追溯到古希腊泰奥弗拉斯托斯的著作《论岩石》Peri，Lithon）在古罗马时期，老普林尼详细的描述了常用的一些矿物和金属，还正确的解释了琥珀的来源。\n一些现代学者（如）认为，现代地质学开始于中世纪伊斯兰世界。比鲁尼就是最早的之一，他的著作有最早的叙述的文章，提出了印度次大陆曾经是海洋的假设。伊斯兰学者伊本·西那对山脉的形成，地震的原因，以及其他一些现代地理学的论题给出了详细的解释，这些内容为日后地质学的发展提供了基础。在中国，博学家沈括（1031–1095）提出陆地形成的假说。他在一个离海洋几百公里远的山中的看到，在一个地质地层里有贝壳类生物化石。由此他推论，陆地是由山脉的侵蚀和淤泥的沉积所形成的。\n很早以前，地质学的知识比较零星分散。关于这方面的知识，如从地中开采金属、黏土、煤和盐的一些知识，早已为矿工和有关的人们所知晓，而自然哲学家们则大都脱离这些实践，独立形成自己的思辨性的地质理论。\n地质学在18世纪开始成为一门独立的科学，并在19世纪早期达到成熟阶段。\n1790年至1830年这一段时期被称为“地质学的英雄时代”在这个时期，在考察岩层顺序以及岩层所含矿物和化石上，人们做了大量工作。在现阶段，由于观察、研究条件的限制，主要以岩石圈为研究对象，并涉及水圈、大气圈、生物圈和岩石圈下更深的部位，以及涉及其他行星和卫星的太空地质学。\n。工作方法的一大进步表现在用根据化石内容来进行岩层分

In [17]:
def gen():
    for txt in items:
        yield {'text': txt}

dataset = Dataset.from_generator(gen, cache_dir='.cache', keep_in_memory=True)

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
eos_token_id = tokenizer.eos_token_id
def txt_to_id_map(samples: dict, max_len: int, stride: int, tokenizer: int, ids_dtype: np.dtype, np) -> dict:

    batch_txt = samples['text']
    eos_token_id = tokenizer.eos_token_id
    encoded = tokenizer(
                        batch_txt, 
                        max_length=max_len, 
                        truncation=True, 
                        stride=stride,                      # 相邻两行保持stride个重复的token
                        return_overflowing_tokens=True,     #返回被截断的数据
                        return_token_type_ids=False, 
                        return_offsets_mapping=False, 
                        return_attention_mask=False,
                    )
            
    input_ids = encoded['input_ids']
    overflow_map = encoded['overflow_to_sample_mapping']

    # 获取每个doc的最后一行
    last_line_indexs = []
    for idx in range(len(overflow_map) - 1):
        # 在分割处的id不一样
        if overflow_map[idx] != overflow_map[idx + 1]:
            last_line_indexs.append(idx) 

    # 添加最后一个doc的最后一行
    last_line_indexs.append(len(overflow_map) - 1)
    
    # 仅在doc的最后添加eos id，如果最后一行长度为max_length，eos id直接覆盖最后一个token id
    for last_idx in last_line_indexs:
        if len(input_ids[last_idx]) == max_len:
            input_ids[last_idx][-1] = eos_token_id
        else:
            input_ids[last_idx] += [eos_token_id]

    outputs = [np.array(item, dtype=ids_dtype) for item in input_ids]

    return {
            "input_ids": outputs
        }

In [ ]:
max_len, stride = 320, 0
ds = dataset.map(txt_to_id_map, fn_kwargs={'max_len': max_len, 'stride': stride, 'tokenizer': tokenizer, 'ids_dtype': ids_dtype, 'np': np}, batched=True, batch_size=1024, remove_columns=dataset.column_names, num_proc=6)

In [26]:
ds

Dataset({
    features: ['input_ids'],
    num_rows: 2079237
})

In [ ]:
ds.save_to_disk('./data/wiki')

In [5]:
def cut_with_end_pun(txt: str, max_len: int) -> str:
    '''
    截断文本，超过最大长度的，从最后一个结束标点符号截断
    '''
    if len(txt) <= max_len:
        return txt 

    # 从 max_len 开始找最后一个句号，叹号
    i = max_len
    while i >= 0 and txt[i] not in ('。', '！'):
        i -= 1
    
    end = max_len if i <= 0 else i + 1
    txt = ''.join(txt[0: end])

    return txt

将Wiki数据合并为长度固定的行

In [18]:
def split_txt_cropus_to_chunk_data(texts: list[str], batch_size: int=512 ** 2, max_len: int=320, window_size: int = 2) -> list[str]:
    
    buffer, buffer_len = [], 0
    chunk_data = []

    for i, line in enumerate(texts):
        buffer_len += len(line)
        buffer.append(line)

        if buffer_len >= batch_size or i == len(texts) - 1:
            buffer_txt = ''.join(buffer)
            
            # - window_size为滑动窗口，这样每个窗口都包含有window_size个上文
            for i in range(0, len(buffer_txt), max_len - window_size):

                chunk_data.append(''.join(buffer_txt[i: i + max_len]))
            
            buffer, buffer_len = [], 0
    
    return chunk_data

In [19]:
chunk_data = split_txt_cropus_to_chunk_data(items)
print(len(chunk_data))

2029501


In [10]:
chunk_data[0: 3]

['数学：数学，是研究数量、结构以及空间等概念及其变化的一门学科，属于形式科学的一种。数学利用抽象化和逻辑推理，从计数、计算、量度、对物体形状及运动的观察发展而成。数学家们拓展这些概念，以公式化新的猜想，以及从选定的公理及定义出发，严谨地推导出一些定理。\n基础数学的知识与运用是生活中不可或缺的一环。对数学基本概念的完善，早在古埃及、美索不达米亚及古印度历史上的古代数学文本便可观见，而在古希腊那里有更为严谨的处理。从那时开始，数学的发展便持续不断地小幅进展，至16世纪的文艺复兴时期，因为新的科学发现和数学革新两者的交互，致使数学的加速发展，直至今日。数学并成为许多国家及地区的教育中的一部分。\n数学在许多领域都有应用，包括科学、工程、医学、',
 '学、经济学和金融学等。数学对这些领域的应用通常被称为应用数学，有时亦会激起新的数学发现，并导致全新学科的发展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。数学家也研究纯粹数学，就是数学本身的实质性内容，而不以任何实际应用为目标。许多研究虽然以纯粹数学开始，但其过程中也发现许多可用之处。\n词源。\n西方语言中“数学”一词源自于古希腊语的（其有“学习”学问”科学”还有个较狭义且技术性的意思－"数学研究"即使在其语源内。其形容词（意思为"和学习有关的"或"用功的"亦会被用来指"数学的"其在英语中表面上的复数形式，及在法语中的表面复数形式\'可溯至拉丁文的中性复数\'由西塞罗译自希腊文复数（此一希腊语被亚里士多德',
 '多德拿来指"万物皆数"的概念。\n汉字表示的"数学"一词大约产生于中国宋元时期。多指象数之学，但有时也含有今天上的数学意义，例如，秦九韶的《数学九章》永乐大典》记，即《数书九章》也被宋代周密所著的《癸辛杂识》记为《数学大略》数学通轨》明代柯尚迁著）数学钥》清代杜知耕著）数学拾遗》清代丁取忠撰）直到1939年，经过中国数学名词审查委员会研究“算学”与“数学”两词的使用状况后，确认以“数学”表示今天意义上的数学含义。\n历史。\n数学有着久远的历史。它被认为起源于人类早期的生产活动：中国古代的六艺之一就有"数"数学一词在西方有希腊语词源（mathematikós意思是“学问的基础”源于（máthema，科学，知识，学问”\n史前的人类就已尝试用']

In [11]:
tb = pa.Table.from_arrays([chunk_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/wiki_chunk_320_2.2M.parquet', row_group_size=50000, data_page_size=50000, )

In [12]:
# wiki_samples = np.random.choice(chunk_data, size=100_0000).tolist()
wiki_samples = chunk_data[0: 10_0000]

In [13]:
len(wiki_samples)

100000

In [14]:
wiki_samples[-5:]

['台御史杨开鼎的倡议下，于该年十月重建孔庙，于乾隆十六年（1751年）三月完工，而此次工程中大成殿与两庑的规模倍前，格局被记载在隔年王必昌编纂的《重修台湾县志》所附之府学宫图中。然后到了乾隆四十二年（1777年）知府蒋元枢又加以整建，主要是在东大成坊外增建了泮宫坊，大成殿西边增设了府学署，为台南孔庙规模最大的时期，其格局载于《重建台郡各建筑图说》之重建台湾府学图与现存于明伦堂里的台湾府学全图碑上。然而后来孔庙却因为嘉庆与同治年间的地震等因素而逐渐毁损。\n据日人山田孝使《台南圣庙考》载，在同治元年（1862年）台湾发生大地震，大成殿两庑有倾圯者，修之"本年元月及五月台湾发生两次地震，特别是五月大地震，使清政府下诏免征田赋。同年官民捐银千',
 '银千余两。然而比起道光十五年修复工程所费金额的规模，此次规模很小。\n光绪十三年（1887年）台湾建省。置台湾、台北、台南三府，将原台湾府改称"台南府"但直到光绪十四年（1888年）才将"台湾府学"改称为"台南府学"\n日治时期。\n自同治年间至日本领台前，有三十余年台南孔子庙皆无整修，以至于蠹蚀严重。乙末（1895年）改隶后，进入日治时期。因各地抗日义军风起云涌，日军将"南进讨伐军驻屯所"设于台南孔子庙内。并在东大成坊设立"台南舍营病院"作为军医院所用。直到抗日义军声势较弱后，将台南孔子庙改为"台湾民政支所职员宿舍"明治二十九年（1896年）11月4日，在此开办"台南国语传习所"\n明治三十年（1897年）前清举人蔡国琳与商朝凤、许廷光等',
 '光等人谋议，由台南孔子庙与附属乐局的田园租金收入，共一千二百九十圆来修葺大成殿及其他建物，并修补礼乐器。然而在明治三十一年（1898年）8月，台湾总督府发布"公学校令"和"公学校规则"在台南孔庙设立"台南公学校"提供台湾人小学程度日文教育，明治三十七年（1904年）更名为"台南第一公学校"当时校舍的使用，除了大成殿、文昌阁及节孝祠外，其余皆作为学校使用。事实上，大成殿亦被充当教室使用。大正五年（1916年）孔庙计划整修，所以学校于隔年（1917年）8月迁于南大附小树林街现址\n\n大正五年（1916年）4月，中华民国福建省指派代表张遵旭来台参观台湾劝业会。将访台18日的见闻撰成《台湾游记》即纪录参观台南孔子庙的境况：现在孔庙改为公学校（',
 '校（公学校，专为教育台湾人而设）

##  1.2 处理百度百科数据库

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson
from unicodedata import normalize

In [2]:
def split_txt_cropus_to_chunk_data(texts: list[str], batch_size: int=512 ** 2, max_len: int=320, window_size: int = 2) -> list[str]:
    
    buffer, buffer_len = [], 0
    chunk_data = []

    for i, line in enumerate(texts):
        buffer_len += len(line)
        buffer.append(line)

        if buffer_len >= batch_size or i == len(texts) - 1:
            buffer_txt = ''.join(buffer)
            
            # - window_size为滑动窗口，这样每个窗口都包含有window_size个上文
            for i in range(0, len(buffer_txt), max_len - window_size):

                chunk_data.append(''.join(buffer_txt[i: i + max_len]))
            
            buffer, buffer_len = [], 0
    
    return chunk_data

In [3]:
bd_baike_563w_file = './data/563w_baidubaike.json'
baike_items = []
eos_token = '[EOS]' 
max_len = 320
batch_size, batch_cnt = 200_0000, 0

In [4]:
with open(bd_baike_563w_file, 'r', encoding='utf-8') as f:
    line = f.readline()
    line = normalize('NFKC', line)
    item = ujson.loads(line)
    print(item)

{'title': '红色食品', 'summary': '红色食品是指食品为红色、橙红色或棕红色的食品。科学家认为,多吃些红色食品可预防感冒。红色食品有红柿椒、西红柿、胡萝卜、红心白薯、红果(山楂)、红苹果、草莓、红枣、老南瓜、红米、柿子等。 有治疗缺铁性贫血和缓解疲劳的作用,对乳腺癌等肿瘤疾病有防治作用,给人以兴奋感,有增加食欲,光洁皮肤,增强表皮细胞再生和防止皮肤衰老,预防感冒等作用。', 'sections': [{'title': '简介', 'content': '红色食品富含番茄红素、胡萝卜素、铁和部分氨基酸,是优质蛋白质、碳水化合物、膳食纤维、B族维生素和多种无机盐的重要来源,可以弥补粳米、白面中的营养缺失。经常食用红色食品,可以进一步提高对主食中营养的利用率,山植等食品还有治疗癌症的功效。被称为“红色生力军。”营养学家认为,红色蔬果最典型的优势在于它们都是富含天然铁质的食物,例如我们常吃的樱桃、大枣等都是贫血患者的天然良药,也适合女性经期失血后的滋补。所以,红色蔬果,女人尽可放心多吃。红色食品中还含有致病微生物的“杀手”——巨噬细胞,可以有效地抵御感冒病毒等微生物,增强人体抵抗感冒的能力。  红色食品\n在所有的果蔬当中,名声最好的莫过于苹果。西方有“One apple a day,keeps the doctors away.”的说法,因为苹果性情温和,含有各种维生素和微量元素,是所有的水果中最接近完美的一个。\n还有一种说法:红色食品是相对于绿色食品而言的,指对人体有害的食品,如各种有毒有害、腐败变质、添加非食用物质的食品。红色食品危害人体健康乃至生命安全,对人体健康亮起了红灯,应当大力查处。'}, {'title': '作用', 'content': '这些食品中富含β-胡萝卜素和维生素A,对孩子上皮组织和呼吸道粘膜有很强的保护作用,可提高预防感冒的能力。\n假如你生来体质较弱,易受感冒病毒的困扰,或者已经被感冒缠上了,红色食品会助你一臂之力,天生具有促进人体健康卫士之一的巨噬细胞活力的功能,巨噬细胞乃是感冒病毒等致病微生物的“杀手”,其活力增强了,感冒病毒自然难以在人体内立足,更谈不上生长繁殖了。至于颜色较辣椒稍浅一些的胡萝卜,所含的胡萝卜素可在体内转化为维生素A,发挥护卫人体上皮组织如呼吸道黏膜的作用,常食之同样可以增强人体抗御感冒的能力

In [5]:
with open(bd_baike_563w_file, 'r', encoding='utf-8') as f:

    def process_none(s: str) -> str:
        if s: return s
        return ''
    
    while True:
        line = f.readline()
        if not line: break

        item = ujson.loads(line)
        cur_txt, cur_len = [], 0

        if not item['title']: continue

        temp_txt = f"{item['title']}：{process_none(item['summary'])}"
        
        cur_len += len(temp_txt)
        cur_txt.append(temp_txt)

        for section in item['sections']:

            # 太长的截断不要了
            if cur_len > max_len:
                break
            
            title = f"{section['title']}：" if section['title'] else ""
            temp_txt = f"{title}{process_none(section['content'])}"
            
            cur_len += len(temp_txt)
            cur_txt.append(temp_txt)
        
        # normalize 处理\u3000 \xa0，全角转半角
        temp_txt =  normalize('NFKC', ''.join(cur_txt))

        if len(temp_txt) > max_len:
            # 从 max_len 开始找第一个句号，叹号
            n, i = len(temp_txt), max_len
            while i < n and temp_txt[i] not in ('。', '！'):
                i += 1
            temp_txt = ''.join(temp_txt[0: i + 1])

        # 添加 eos token
        temp_txt = f"{temp_txt}{eos_token}"
        
        baike_items.append( temp_txt )

        if len(baike_items) % batch_size == 0:

            chunk_data = split_txt_cropus_to_chunk_data(baike_items)
            tb = pa.Table.from_arrays([chunk_data], names=['text'])

            file_name = f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet'
            pq.write_table(table=tb, where=file_name, row_group_size=50000, )

            print(f"save to {file_name}")

            batch_cnt += 1
            baike_items = []

    if len(baike_items) > 0:
        chunk_data = split_txt_cropus_to_chunk_data(baike_items)
        tb = pa.Table.from_arrays([chunk_data], names=['text'])

        file_name = f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet'
        pq.write_table(table=tb, where=file_name, row_group_size=50000, )

        print(f"save to {file_name}")

        batch_cnt += 1
        baike_items = []

save to ./data/baike_chunk_320_5.6M_0.parquet
save to ./data/baike_chunk_320_5.6M_1.parquet
save to ./data/baike_chunk_320_5.6M_2.parquet


In [6]:
file_list = [
    f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet' for batch_cnt in range(3)
]

line_cnt = 0 
for file in file_list:
    pf = pq.read_table(file)
    line_cnt += pf.num_rows

print(f"bake all lines: {line_cnt}")

bake all lines: 5195460


In [7]:
chunk_data[20: 25]

['连市委办公厅、大连市人民政府办公厅关于印发<旅顺口区人民政府机构改革方案>的通知》(大委办发【2009】29号),设立大连市旅顺口区安全生产监督管理局,正处级建制,为区政府工作部门。[1]主要职责:(一)贯彻落实国家、省、市关于安全生产方面的方针、政策、法律、法规;拟订全区安全生产工作政策和规划;制定发布各行业综合性安全生产规程;指导、协调和监督全区安全生产工作;定期分析和预测全区安全生产形势;负责发布全区安全生产信息;协调解决安全生产中的重大问题。 \n(二)承担全区安全生产综合监督管理责任,依法行使综合监督管理职权;指导协调、监督检查区政府有关部门和各街道安全生产工作;负责区政府安全生产目标管理考核工作的具体组织实施,监督考核并通',
 '并通报安全生产控制指标执行情况,监督事故查处和责任追究落实情况。[EOS]陶令秫:táo lìng shú ㄊㄠˊ ㄌㄧㄥˋ ㄕㄨˊ 陶令秫 [EOS]炝炒虾球西兰花:原料::青虾250克、料酒1汤勺、干淀粉2汤勺、橄榄油1汤勺、盐1茶匙、生抽1汤勺。 做法::1、青虾去头去壳去虾腺,从虾背处破刀。 \n2、清理好的虾仁放在碗中,撒上1/2茶匙底盐腌制入味后,用厨房用纸吸去虾仁身上多余的水分。放入淀粉用手抓几下虾仁,使淀粉上劲包裹住虾仁。上浆后的虾仁静止5-10分钟。 \n3、煮锅中倒入适量清水烧沸,淋上少许橄榄油,下入西兰花焯烫30秒捞出。 \n4、放在冷水中浸泡片刻捞出沥干备用。 \n5、炒锅加热,倒入适量的炒菜油。约三成热的时候下入虾',
 '入虾仁滑炒,微微变色后捞出备用。 \n6、炒锅倒入底油,约6成热左右,葱炝锅放入西兰花,大火快手翻炒,放入滑过油的虾仁,顺着锅边烹入一勺料酒,淋上生抽,出锅前再加入1/2勺盐调味后即可起锅装盘。[EOS]刘道辰:刘道辰[1]:男,1964年出生,聊城大学副教授,副院长兼办公室主任,主要从事自然地理学等方面的教学、科研工作;在核心期刊发表科研论文20余篇,参编著作、教材5部,主持参与科研项目10余项。 [EOS]邓慧平:邓慧平[1]:男,1962年出生,理学博士,1996年博士后出站,聊城大学教授;从事气象气候学等方面的教学科研工作,发表科研论文30余篇,2004年10月至2006年11月于美国进行科研合作。 [EOS]李仰东:人物简介',
 '简介:[1]\n李仰东,男,中共党

## 1.3 处理bell指令数据
尝试在预训练阶段加入prompt指令数据，就是尝试在预训练解决加加入部分Sft数据

In [2]:
train_data = []
eval_data = []
eval_size = 1_0000
max_len = 400
root = 'D:/GitHub/ChatLM-mini-Chinese/data/raw_data'

In [3]:
with open(root + '/bell_open_source/train_3.5M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        if len(item['conversations']) != 2: continue

        conversation = item['conversations']
        txt = ''
        if conversation[0]['from'] =='human':
            txt = f"{conversation[0]['value']}\n{conversation[1]['value']}"
        else:
            txt = f"{conversation[1]['value']}\n{conversation[0]['value']}"
        
         # 收集测试数据
        if len(txt) >= max_len and len(txt) < max_len + 8 and len(eval_data) < eval_size and np.random.rand() <= 0.12:
            eval_data.append(txt)
            continue
            

        if len(txt) >= max_len: continue
        train_data.append(txt)

In [4]:
print(len(eval_data), len(train_data))

5429 1084177


In [5]:

for file in [root + '/bell_open_source/train_2M_CN.json',  root + '/bell_open_source/Belle_open_source_1M.json']:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            item = ujson.loads(line)

            if item['input'].strip() != '':
                txt = f"{item['instruction']}\n{item['input']}\n{item['output']}"
            else:
                txt = f"{item['instruction']}\n{item['output']}"

            # 收集测试数据
            if len(txt) >= max_len and len(txt) < max_len + 8 and len(eval_data) < eval_size and np.random.rand() > 0.75:
                eval_data.append(txt)
                continue
            
            if len(txt) == 0 or len(txt) >= max_len: continue
            train_data.append(
                    txt
            )

In [6]:
print(len(eval_data), len(train_data))

10000 3150704


In [7]:
print(train_data[0:5])


['请输入一段描述有关海洋保护的情景对话。\nPerson A: 哇，这个海滩真美啊！\nPerson B: 是啊，可惜现在海洋污染越来越严重了。\nPerson A: 对啊，以前来海边能看到很多海星、海螺等生物，现在好难看到了。\nPerson B: 这些可爱的生物其实都是海洋生态系统的重要组成部分，如果它们消失了，整个生态系统都会受影响。\nPerson A: 我们该怎么做才能保护海洋呢？\nPerson B: 首先，我们要遵守海滩的相关规定，不要随便乱扔垃圾。其次，减少使用一次性塑料制品，比如塑料袋、吸管等，这些东西会进入海洋，危害海洋生物。最后，我们可以积极参与各种海洋保护组织，呼吁更多人加入到保护海洋的行动中来。\nPerson A: 听起来保护海洋是一项重要的任务，我们一定要做好自己的份内工作。\nPerson B: 是的，每个人都能为保护海洋贡献自己的一份力量。', '为我寻找5个值得信任的汽车保险公司\n1. State Farm\n2. Geico\n3. Allstate\n4. Progressive\n5. Farmers Insurance', 'A公司去年亏损了500万美元，今年净利润增长了50%，今年的净利润是多少？\n今年净利润为750万美元', '介绍最适合初学者的编程语言\nPython是最适合初学者学习的编程语言之一。Python具有以下优点：\n1. 简单易学：Python拥有简单易懂的语法和基本代码结构，可以使初学者更快地掌握编程基础。\n2. 应用广泛：Python可以用于很多不同的任务和领域，例如数据科学、人工智能、机器学习、网络编程等。\n3. 强大的社区支持：Python具有庞大的开发者社区，可以获得丰富的资源和支持。\n4. 多资源学习：Python的学习资源丰富，包括图书、视频、课程等。\n总之，对于初学者来说，Python是一门非常好的编程语言，它容易入门，应用广泛，有强大的社区支持和丰富的学习资源。', '以下是一道小学数学题： \n有一条直线，上面有 6 只蚂蚁，其中 4 只向左走，2 只向右走。问：向右走的蚂蚁数量占比是多少？\nA. 25%\nB. 33.3%\nC. 50%\nD. 66.6%\n回答该问题。\n向右走的蚂蚁数量占比为两只蚂蚁中向右走的蚂蚁数量与总蚂蚁数量的比值。\n先算出总蚂蚁数量：

In [8]:
tb = pa.Table.from_arrays([train_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where=f'./data/bell_pretrain_{max_len}_3M.parquet', row_group_size=20480, data_page_size=20480, )

In [9]:
tb = pa.Table.from_arrays([eval_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where=f'./data/pretrain_eval_{max_len}_1w.parquet', row_group_size=20480, data_page_size=20480, )

# 2. 处理sft阶段数据

In [6]:
lines = []
with open('./data/sft_0.8M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        txt = f"{item['instruction']}{item['output']}"
        
        if len(txt) == 0 or len(txt) >= 320: continue
        lines.append(
                item
        )

In [7]:
print(len(lines))

726475


In [8]:
tb = pa.Table.from_pylist(lines)
# compression='GZIP'
pq.write_table(table=tb, where='./data/sft_train_data.parquet', row_group_size=20480, data_page_size=20480, )

## 统计 token数量

In [8]:
from pyarrow import parquet as pq
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm

In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('./model_save/tokenizer/')

In [10]:
# 字符数量
files = [
    './data/baike_chunk_320_5.6M_0.parquet', 
    './data/baike_chunk_320_5.6M_1.parquet', 
    './data/baike_chunk_320_5.6M_2.parquet', 
    './data/bell_pretrain_400_3M.parquet',
    # './data/pretrain_eval_400_1w.parquet',
]

total_char = 0
for file in files: 
    pf = pq.read_table(file)
    for row in pf['text']:
        total_char += len(row.as_py())

In [11]:
print(total_char)

2247061858


In [13]:
total_token = 0
buffer = []
for file in files: 
    pf = pq.read_table(file)
    n = pf.num_rows
    for i, row in tqdm(enumerate(pf['text']), total=n):
        buffer.append(row.as_py())

        if len(buffer) >= 10000 or i == n - 1:
            input_ids = tokenizer(buffer, return_attention_mask=False)['input_ids']
            
            total_token += sum([len(item) for item in input_ids])
            buffer = []

if len(buffer) > 0:
    input_ids = tokenizer(buffer, return_attention_mask=False)['input_ids']
    
    total_token += sum([len(item) for item in input_ids])
    buffer = []

100%|██████████| 3150704/3150704 [01:55<00:00, 27381.06it/s]


In [14]:
print(total_token)

1445852913
